In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [2]:
X_filepath = '/home/Chess-Piece-Threat-Detection/dataset/int-boards/data.npy'
y_filepath = '/home/Chess-Piece-Threat-Detection/dataset/int-boards/data_labels.npy'

In [3]:
y = np.load(y_filepath)
X = np.load(X_filepath)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
X_train = X_train / 5
X_test = X_test / 5

In [6]:
#validation is first 10k
X_val = X_test[:10000]
#training is 10k+
X_test = X_test[10000:]

y_val = y_test[:10000]
y_test = y_test[10000:]
print('X_train:', X_train.shape)
print('X_test:\t', X_test.shape)
print('X_val:\t', X_val.shape)

X_train: (80000, 8, 8)
X_test:	 (10000, 8, 8)
X_val:	 (10000, 8, 8)


In [7]:
X_train = X_train.reshape(80000, 8, 8, 1)
X_test = X_test.reshape(10000, 8, 8, 1)
X_val = X_val.reshape(10000, 8, 8, 1)

In [8]:
model = keras.Sequential()
model.add(Conv2D(128, input_shape=(8, 8, 1), padding='same', kernel_size=4, activation='LeakyReLU'))
model.add(Conv2D(128, kernel_size=4, padding='same', activation='LeakyReLU'))
model.add(Conv2D(32, kernel_size=4, padding='same', activation='LeakyReLU'))
model.add(Flatten())
#model.add(Dense(1024, activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [9]:
es = EarlyStopping(monitor='val_loss', patience=5)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 8, 8, 128)         2176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 128)         262272    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          65568     
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 12294     
Total params: 342,310
Trainable params: 342,310
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=256, verbose=1, validation_data=(X_val, y_val), callbacks=[es])

Epoch 1/100
313/313 [==============================] - 148s 395ms/step - loss: 0.5541 - accuracy: 0.1773 - val_loss: 0.5348 - val_accuracy: 0.1999
Epoch 2/100
313/313 [==============================] - 119s 381ms/step - loss: 0.5261 - accuracy: 0.2059 - val_loss: 0.5236 - val_accuracy: 0.2215
Epoch 3/100
313/313 [==============================] - 120s 384ms/step - loss: 0.5038 - accuracy: 0.2281 - val_loss: 0.5098 - val_accuracy: 0.3236
Epoch 4/100
313/313 [==============================] - 120s 383ms/step - loss: 0.4894 - accuracy: 0.2436 - val_loss: 0.4912 - val_accuracy: 0.3413
Epoch 5/100
186/313 [================>.............] - ETA: 46s - loss: 0.4811 - accuracy: 0.2576

In [ ]:
model.evaluate(X_test, y_test, batch_size=175, verbose=1)

In [ ]:
predictions = model.predict(X_test)
print(predictions[0])
print(y_test[0])

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss Value')